In [22]:
import pandas as pd
import numpy as np
import datetime as dt
import statsmodels.api as sm
#
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed

import tools
import importlib
importlib.reload(tools)

<module 'tools' from '/home/keithyamkf/jupyter-notebook/slack-trading/tools.py'>

### Download data from AWS S3

In [23]:
s3_bucket = 'slack-trading'
local_path = '../data'
tools.download_data_from_s3(s3_bucket, local_path)

### CPU Count

In [24]:
cpu_count()

4

### Begin

In [25]:
num_of_years = 1
start = dt.date.today() - dt.timedelta(days = int(365.25*num_of_years))
end = dt.date.today()

In [26]:
# dataset = pd.read_csv('../data/all_stock_close.csv')
dataset = pd.read_csv('../data/all_stock_close_cleaned.csv')
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset.set_index('Date', drop=True, inplace=True)
dataset.head(3)

,0001.HK,0002.HK,0003.HK,0004.HK,0005.HK,0006.HK,0007.HK,0008.HK,0010.HK,0011.HK,...,6868.HK,6869.HK,6878.HK,6881.HK,6886.HK,6889.HK,6890.HK,6893.HK,6896.HK,9988.HK
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-11,66.068962,75.704773,13.765183,17.992689,57.830746,52.005646,0.84,4.296173,18.745403,149.965576,...,5.792519,13.036684,0.43,4.043017,12.284131,9.617226,1.0494,0.65,1.102041,197.600006
2019-12-12,67.205635,76.889137,13.857778,18.326250,58.378670,52.524284,0.84,4.324192,18.823429,151.492905,...,5.782881,13.485552,0.44,4.023580,12.225635,9.617226,1.0494,0.62,1.102041,200.199997


In [27]:
prevday_returns = np.log(dataset.shift(1)/dataset.shift(2))
today_returns = np.log(dataset/dataset.shift(1))
#
stocks_prev = prevday_returns.columns.to_list()
stocks_today = today_returns.columns.to_list()
#
new_col = ['{}.prev'.format(c) for c in stocks_prev]
prevday_returns.columns = new_col
#
comb_df = pd.concat([prevday_returns, today_returns], axis=1)

### Fina the NaN in comb_df

In [28]:
prev_col = [x for x in comb_df.columns.to_list() if '.prev' in x]

In [29]:
comb_df[prev_col].dropna(axis=0, how='all')

,0001.HK.prev,0002.HK.prev,0003.HK.prev,0004.HK.prev,0005.HK.prev,0006.HK.prev,0007.HK.prev,0008.HK.prev,0010.HK.prev,0011.HK.prev,...,6868.HK.prev,6869.HK.prev,6878.HK.prev,6881.HK.prev,6886.HK.prev,6889.HK.prev,6890.HK.prev,6893.HK.prev,6896.HK.prev,9988.HK.prev
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-13,0.017058,0.015523,0.006704,0.018369,0.009430,0.009923,0.000000,0.006501,0.004154,0.010133,...,-0.001665,0.033852,0.022989,-0.004819,-0.004773,0.000000,0.000000,-0.047253,0.000000,0.013072
2019-12-16,0.045462,0.017105,0.011958,0.028498,0.029426,0.008047,0.011835,-0.004329,0.014404,0.026735,...,0.000000,-0.005806,0.011300,0.042560,0.034486,0.005877,0.000000,0.077558,0.000000,0.008951
2019-12-17,0.006711,0.000605,0.001324,-0.006263,-0.005816,0.018527,-0.023811,0.006486,-0.011300,-0.009245,...,-0.008368,0.034338,0.000000,0.000000,-0.001542,0.003899,0.000000,0.029414,-0.008368,-0.004963
2019-12-18,0.009321,-0.005463,0.003947,0.010417,0.011599,-0.003503,0.000000,0.002153,-0.002068,0.011084,...,0.008368,0.000000,-0.011300,0.043042,0.037855,0.000000,0.000000,0.000000,0.008368,0.013834
2019-12-19,-0.019405,0.010294,-0.006589,0.003104,-0.001649,-0.003515,0.000000,0.004292,-0.002072,-0.006143,...,0.000000,-0.019887,-0.011429,0.000000,-0.002976,-0.003899,-0.009479,0.042560,-0.016807,-0.005906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-14,-0.002689,-0.002110,-0.005106,-0.008265,-0.005984,-0.004848,-0.037388,0.012579,0.000000,0.005831,...,0.028121,-0.001974,0.027029,-0.002148,0.003378,0.014056,0.000000,0.000000,0.000000,0.011678
2020-12-15,-0.011738,-0.002115,0.000000,0.005173,-0.007229,-0.006094,-0.009569,0.004158,0.004167,-0.005100,...,0.005186,-0.005946,-0.013423,0.000000,-0.017007,-0.016159,0.000000,-0.019418,0.007018,-0.026668
2020-12-16,-0.023898,-0.004952,-0.017212,0.000000,-0.008500,-0.009828,0.009569,-0.002077,-0.008351,0.003646,...,0.008584,-0.025165,0.000000,-0.017354,-0.015558,0.013690,-0.012739,0.038466,-0.014085,-0.022509


In [30]:
comb_df.loc['2019-12-23', prev_col]

0001.HK.prev    0.013496
0002.HK.prev   -0.003638
0003.HK.prev    0.001329
0004.HK.prev    0.007205
0005.HK.prev   -0.000823
                  ...   
6889.HK.prev   -0.017734
6890.HK.prev    0.000000
6893.HK.prev    0.000000
6896.HK.prev    0.000000
9988.HK.prev    0.010685
Name: 2019-12-23 00:00:00, Length: 1219, dtype: float64

In [31]:
comb_df.isnull().values.any()

True

In [32]:
comb_df.dropna(axis=0, how='all', inplace=True)

In [33]:
comb_df.head(3)

,0001.HK.prev,0002.HK.prev,0003.HK.prev,0004.HK.prev,0005.HK.prev,0006.HK.prev,0007.HK.prev,0008.HK.prev,0010.HK.prev,0011.HK.prev,...,6868.HK,6869.HK,6878.HK,6881.HK,6886.HK,6889.HK,6890.HK,6893.HK,6896.HK,9988.HK
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.001665,0.033852,0.022989,-0.004819,-0.004773,0.000000,0.0,-0.047253,0.000000,0.013072
2019-12-13,0.017058,0.015523,0.006704,0.018369,0.009430,0.009923,0.000000,0.006501,0.004154,0.010133,...,0.000000,-0.005806,0.011300,0.042560,0.034486,0.005877,0.0,0.077558,0.000000,0.008951
2019-12-16,0.045462,0.017105,0.011958,0.028498,0.029426,0.008047,0.011835,-0.004329,0.014404,0.026735,...,-0.008368,0.034338,0.000000,0.000000,-0.001542,0.003899,0.0,0.029414,-0.008368,-0.004963


In [34]:
prevday_returns = comb_df[new_col]
today_returns = comb_df[stocks_today]

In [35]:
def get_linear_regression(pair, prevday_returns, today_returns):
    catstock = pair['prev']
    column = pair['today']
    X = prevday_returns[[catstock]].copy()
    X = sm.add_constant(X)
    y = today_returns[[column]]
    model = sm.OLS(y, X).fit()
    data = {
        'Prev' : catstock,
        'Today' : column,
        'Params' : model.params.iloc[1],
        'TValues' : abs(model.tvalues.iloc[1]),
        'RSquared' : model.rsquared
    }
    return data

### Compare linear regression for 2 selected stocks

In [ ]:
prev_stock = '6190.HK.prev'
today_stock = '0247.HK'
prevday_returns[prev_stock]
today_returns[today_stock]

In [ ]:
prevday_returns

### Non parallel processing

In [ ]:
%%time
catstocks = new_col
catstocks = ['0008.HK.prev', '0010.HK.prev']
pair_list = []
for catstock in catstocks:
    for column in today_returns.columns:
        if column not in catstock:
            data = {
                'prev' : catstock,
                'today' : column
            }
            pair_list.append(data)
#
results = [get_linear_regression(pair, prevday_returns, today_returns) for pair in pair_list]
result_df = pd.DataFrame(results)
result_df

### Parrallel processing

In [ ]:
%%time
catstocks = new_col
# catstocks = ['0008.HK.prev', '0010.HK.prev']
pair_list = []
for catstock in catstocks:
    for column in today_returns.columns:
        if column not in catstock:
            data = {
                'prev' : catstock,
                'today' : column
            }
            pair_list.append(data)
#
cpu = cpu_count()
print('cpu count = {}'.format(cpu))
executor = Parallel(n_jobs=cpu, backend='multiprocessing')
tasks = (delayed(get_linear_regression)(pair, prevday_returns, today_returns) for pair in pair_list)
results = executor(tasks)
result_df = pd.DataFrame(results)
result_df

In [ ]:
result_df.to_csv('../data/regression_pair.csv')

### Read Regression Result

In [ ]:
result_df = pd.read_csv('../data/regression_pair.csv', index_col=0)

In [ ]:
sorted_df = result_df.sort_values(by='RSquared', ascending=False)
sorted_df.head(10)

### Analyze the Top pair

In [ ]:
stock1 = '6190.HK.prev'
stock2 = '0247.HK'
#
tools.plot_two_stocks(stock1, stock2, prevday_returns, today_returns, start)

### Upload to AWS S3

In [ ]:
s3_bucket = 'slack-trading'
local_path = '../data'
tools.upload_data_to_s3(s3_bucket, local_path)